# Retrieving the Results of Long-Running Tasks #

We prefer Forge to be used in a way which enables rapid, interactive iteration.  However, with many simulators and quantum devices, the delays can be quite long.

Under the hood, when you call an API function, the call is submitted instantly, and we wait for the result.  If the wait is too long, the task will "time out".  The default timeout value can be found as follows:

In [1]:
from qcware import forge
import pprint
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('retrieving_long_task_results.ipynb')
print(f"Default client timeout: {forge.config.client_timeout()} seconds")

Default client timeout: 60 seconds


## What does this number mean? ##

When the client waits for the server, there are two kinds of "wait":

### server_timeout ###

`server_timeout` is how long the server will sit waiting for a backend to return a value before giving up and telling the client it hasn't heard anything.  The default is 10 seconds, and this is not normally something the user should modify.

### client_timeout ###

`client_timeout` is how long the _client_ will wait before giving up and raising an exception.  This is 60 seconds by default, but can be as high as the user awants.


## Should I change the client timeout?  ##

It depends on your use case!  Normally you won't have to, but if you need to wait for a long call, you may want to change `client_timeout` (there often isn't a reason to change `server_timeout`, but you're certainly welcome to, although it must be less than 50 seconds).  

On the other hand, there's nothing wrong with a task timing out; the task will still be executed, and you can retrieve the results at your leisure.

To show this, let's set the client timeout to 0 and call a function such that it times out instantly.  This is a special case--with the client timeout set to zero, the call will _always_ raise an `ApiTimeoutError` exception (and never ask the server for the result) which enables you to start many calls and selectively retrieve their results.

We'll solve a small qubo using a classical QC simulator running a QAOA algorithm, which should take a few seconds.  Forge will throw an `ApiTimeoutError` exception with the call token in the description:

In [2]:
forge.config.set_client_timeout(0)
import qcware.types

Q = {(0, 0): 1, (1, 1): 1, (0, 1): -2, (2, 2): -2, (3, 3): -4, (3, 2): -6}
qubo = qcware.types.optimization.PolynomialObjective(
    polynomial=Q,
    num_variables=4,
    domain='boolean'
)
problem = qcware.types.optimization.BinaryProblem(objective=qubo)

thrown_exception = None
try:
    result = forge.optimization.optimize_binary(instance=problem, backend='qcware/cpu_simulator')
    print(result)
except forge.exceptions.ApiTimeoutError as e:
    thrown_exception = e
    print(thrown_exception)

API Call timed out.
You can retrieve with qcware.api_calls.retrieve_result(call_token='b457af44-1943-45b3-b0bf-610cd016aaa3')
or use the .submit or .call_async forms of the API call.
See the getting started notebook "Retrieving_long_task_results.ipynb" in Forge


That call token represents the ID of the call you made.  You can see the results within your web app interface, but you can also retrieve them for future use using the `retrieve_result` function: the exception contains a long "call token" which can be retrieved from the thrown exception with `thrown_exception.api_call_info['uid']`

## Note! ##

If you run the cell below and get the same exception, the call still hasn't finished running on the server; wait a few seconds and try again!

In [3]:
# we'll pause for a few seconds to let the call complete
import time
time.sleep(5)
# Since we have an exception object from above, we can retrieve the result token using this:
result = forge.api_calls.retrieve_result(thrown_exception.api_call_info['uid'])
# otherwise we would have to use something like
# result = qcware.api_calls.retrieve_result("xxxxxxxx-yyyy-zzzz-xxxx-yyyyyyyyyyyy")
print(result)

Objective value: -12
Solution: [0, 0, 1, 1] (and 1 other equally good solution)


# Submitting jobs without waiting for a result

The above pattern uses the `ApiTimeoutError` exception to denote _exceptional_ flow, because Forge is primarily intended to be an interactive platform with a response time suitable for interactive work.  But sometimes submitting a job and retrieving the result later isn't an exceptional control flow pattern--it's what you intended.  

For that, since trapping and dealing with exceptions is clumsy in that context, we provide a way to `submit` the API call instead of calling it directly, check the status of the call, and retrieve the result.

In [4]:
call_id = forge.optimization.optimize_binary.submit(instance=problem, backend='qcware/cpu_simulator')
print(f"API call submitted with id {call_id}")

# now that we've submitted the call, check its status, wait a bit, and check
# again until it's no longer open
time.sleep(0.5)
status = forge.api_calls.status(call_id)
print(f"Call is {status['status']}")
while status['status'] == 'open':
    time.sleep(1.0)
    status = forge.api_calls.status(call_id)
    
# now retrieve the results!
result = forge.api_calls.retrieve_result(call_id)
print(result)

API call submitted with id f4d441c6-a703-408d-9624-dc64e0d19115


Call is open


Objective value: -12
Solution: [1, 1, 1, 1] (and 1 other equally good solution)


# Submitting jobs and waiting until they return

All that's well and good.  But what if you're willing to wait as long as it takes?

One solution is to simply set the timeout to be a very large value.  A more elegant solution is to use Python's async/await and the `async_` version of your call.  (note: If you're used to Python's await/async, it make look strange to call `await` right here, but within a jupyter notebook, you are working within an existing event loop!)

We provide one method similar to `retrieve_result` which will wait as long as necessary to retrieve a submitted job:

In [5]:
call_id = forge.optimization.optimize_binary.submit(instance=problem, backend='qcware/cpu_simulator')
print(f"API call submitted with id {call_id}")
result = await forge.api_calls.async_retrieve_result(call_id)
print(result)

API call submitted with id ee900958-e9cf-4a27-a0da-b07cb0e83876


Objective value: -12
Solution: [0, 0, 1, 1] (and 1 other equally good solution)


but one can also simply call the base API call as `.call_async` to wait as long as necessary.  This allows the user to create just about whatever workflow they desire.

In [6]:
result = await(forge.optimization.optimize_binary.call_async(instance=problem, backend='qcware/cpu_simulator'))
print(result)

Objective value: -12
Solution: [1, 1, 1, 1] (and 1 other equally good solution)


This allows some rather neat tricks.  For example, suppose you want to see the result of a computation across various backends; you can leverage `asyncio`'s task capabilities to create a bundle of computations that can be submitted at once, calculated more or less in parallel, and then the results can be retrieved all at once when each result has been calculated and returned:

In [7]:
import asyncio
results = await asyncio.gather(
    # we'll solve simultaneously using 'classical' (a  brute-force solver)
    asyncio.create_task(forge.optimization.optimize_binary.call_async(instance=problem, backend='qcware/cpu')),
    # and also 'qcware/cpu_simulator' (a QAOA solver using a classical statevector simulator)
    asyncio.create_task(forge.optimization.optimize_binary.call_async(instance=problem, backend='qcware/cpu_simulator')),
    # and finally using the D-wave annealer
    asyncio.create_task(forge.optimization.optimize_binary.call_async(instance=problem, backend='dwave/2000q')),
    )

import pprint
for result in results:
    pprint.pprint(result.results)

[BinarySample(bitstring=[0, 0, 1, 1], energy=-12, num_occurrences=1),
 BinarySample(bitstring=[1, 1, 1, 1], energy=-12, num_occurrences=1)]
[BinarySample(bitstring=[1, 1, 1, 1], energy=-12, num_occurrences=229),
 BinarySample(bitstring=[0, 0, 1, 1], energy=-12, num_occurrences=217),
 BinarySample(bitstring=[0, 1, 1, 1], energy=-11, num_occurrences=194),
 BinarySample(bitstring=[1, 0, 1, 1], energy=-11, num_occurrences=207),
 BinarySample(bitstring=[1, 1, 0, 1], energy=-4, num_occurrences=16),
 BinarySample(bitstring=[0, 0, 0, 1], energy=-4, num_occurrences=15),
 BinarySample(bitstring=[0, 1, 0, 1], energy=-3, num_occurrences=10),
 BinarySample(bitstring=[1, 0, 0, 1], energy=-3, num_occurrences=12),
 BinarySample(bitstring=[1, 1, 0, 0], energy=0, num_occurrences=24),
 BinarySample(bitstring=[0, 0, 0, 0], energy=0, num_occurrences=23),
 BinarySample(bitstring=[1, 0, 0, 0], energy=1, num_occurrences=34),
 BinarySample(bitstring=[0, 1, 0, 0], energy=1, num_occurrences=19)]
[BinarySample(bi